# 1. Data Preparation

### Loading Packages

In [613]:
import time
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score

import pingouin as pg
from scipy.stats import f_oneway, shapiro, kruskal
import scikit_posthocs as posthocs
from statsmodels.multivariate.manova import MANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from skbio.stats.distance import permanova
from skbio.diversity import beta_diversity
# also install necessary dependencies of the excel plugin of pandas

## Data: Bray et al. (2017)

The Cell Image Library has a “Human U2OS Cell: Compound Cell-Painting Experiment” project data that contains the images of 375 plates in 384-well format (More details: https://www.cellimagelibrary.org/pages/project_20269): 

- The images are of U2OS cells treated with each of over 30,000 known bioactive compounds.
- These cells are labeled with 6 labels that characterize seven organelles (the cell-painting assay).
- The data set is comprised of 988,994 fields of view.
- Each field was imaged in five channels (detection wavelengths), and each channel is stored as a separate, grayscale image file.
- As a result, there are approximately 5 million image files in 16-bit TIFF format.

Bray et al. (2017)’s research uses the raw image data of, and includes highly multiplexed measurements of cellular morphology of the tested compounds from the above Human U2OS Cell research. It includes:

- data files containing morphological features derived from each cell in each image (both at the single-cell level and population-averaged, i.e. per-well level),
- the image analysis workflows that generated the morphological features,
- quality-control metrics are provided as metadata,
- chemical annotations for the applied compound treatments.

- Description and files: http://gigadb.org/dataset/view/id/100351/Files_page/1
- Codebase: https://github.com/gigascience/paper-bray2017/tree/master


### Functions for SQLite Databases

In [390]:
# function for getting a dataframes' column list
def get_columns(df: pd.DataFrame):
    return df.columns.to_list()

# function for moving a column to a new position
def move_column(df, col_name, new_position):
    temp_col = df[col_name]
    df = df.drop(columns=[col_name])
    df.insert(new_position, col_name, temp_col)
    return df

# function for converting a column to numeric
def convert_to_numeric(df):
    for col in df.columns:
        # Attempt to convert the column to numeric, setting errors='ignore' keeps the original data if conversion fails
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

# function for getting the memory size of an object
def memory_size(bytes_size: int) -> str:
    for unit in ['Bytes', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024
    return f"{bytes_size:.2f} PB"

In [391]:
# funtion for retrieving the sql database connector
def get_db_conn(db_path: str):
    try:
        conn = sql.connect(db_path) # connect to the SQLite database
        return conn
    except sql.Error as e:
        print(f"An error occurred while accessing the database: {e}")
        pass

# function for connecting a database object and returning its connector cursor
def get_db_cursor(db_path: str):
    try:
        conn = sql.connect(db_path) # connect to the SQLite database
        cursor = conn.cursor() # create a cursor object to execute SQL queries
        return cursor
    except sql.Error as e:
        print(f"An error occurred while accessing the database: {e}")
        pass

# function for viewing the available database tables
def get_db_tables(cursor: sql.Cursor):
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") # cursor execution for retrieving the list of all tables
        db_tables = cursor.fetchall() # fetch the command output in a list object

        # view the list of tables
        temp = []
        for table in db_tables:
            temp.append(table[0])
        
        db_tables = temp
        return db_tables
    
    except sql.Error as e:
        print(f"An error occurred while accessing the database: {e}")
        return []

# function for extracting all available table columns in a dictionary
def get_db_columns(cursor: sql.Cursor):
    db_columns = {}
    
    for table in get_db_tables(cursor=cursor):
        cursor.execute(f"PRAGMA table_info({table});") # cursor call for retrieving table information
        cursor_output = cursor.fetchall() # cursor fetch operation to retrieve the SQL command output
        columns = [col[1] for col in cursor_output]
        db_columns[table] = columns

    for table in db_columns:
        print(f'{table}: {len(db_columns[table])} Columns')
        print(f'{table}: {db_columns[table]} Columns\n')

    return db_columns

# function for printing out the database table sizes
def get_db_size(cursor: sql.Cursor):
    for table in get_db_tables(cursor=cursor):
        # cursor call for executing SQL command 
        cursor.execute(f"SELECT COUNT(*) FROM {table};") 
        # fetch the first element of the cursor call output
        row_count = cursor.fetchone()[0] 
        
        # cursor command for retrieving column information
        cursor.execute(f"PRAGMA table_info({table});") 
        column_count = len(cursor.fetchall())
        
        # print the dimension and size information of each table
        print(f"{table} | Rows: {row_count} | Columns: {column_count} | Datapoints: {(row_count * column_count / 1000000):.1f}M")

def add_id_info(dataframes: dict):
    for table in dataframes.keys():
        if table == 'Image':
            # Split the FileName column into well information and picture number
            dataframes[table]['WellID'] = dataframes[table]['Image_FileName_CellOutlines'].str.split('--').str[0].str.split('_').str[0]
            dataframes[table]['FieldID'] = dataframes[table]['Image_FileName_CellOutlines'].str.split('--').str[0].str.split('_').str[1]
        else:
            # adding well ID and field ID to other tables
            dataframes[table] = dataframes[table].merge(dataframes['Image'][['TableNumber', 'WellID', 'FieldID']], on='TableNumber', how='left')
    
    return dataframes

def fix_columns(dataframes: dict):
    for table in dataframes:
        # change TableNumber to tableID in all of the tables
        dataframes[table].rename(columns={'TableNumber': 'TableID'}, inplace=True)

        # fix the order of the newly added columns
        dataframes[table] = move_column(dataframes[table], 'PlateID', 1)
        dataframes[table] = move_column(dataframes[table], 'WellID', 2)
        dataframes[table] = move_column(dataframes[table], 'FieldID', 3)

        # convert all numeric columns to float
        if table in ['Cells', 'Cytoplasm', 'Nuclei']:
            for eachColumn in dataframes[table].columns:
                if 'AreaShape' in eachColumn:
                    try:
                        # replace 'nan' strings with np.nan
                        dataframes[table][eachColumn] = dataframes[table][eachColumn].replace('nan', np.nan)
                        # attempt to convert the column to numeric type
                        dataframes[table][eachColumn] = pd.to_numeric(dataframes[table][eachColumn])
                        # print(f"Converted column '{eachColumn}' in table '{table}' to numeric.")
                    except ValueError:
                        # if conversion fails, leave the column unchanged
                        print(f"Could not convert column '{eachColumn}' in table '{table}'. Leaving it unchanged.")
    return dataframes

### Loading SQLite Databases

In [392]:
# list of plate IDs being extracted
analysis_mode = 'test'
plate_list = {'live': [24279, 24280, 24293, 24294, 24295, 24296, 24300, 24301, 24302, 24303],
              'test': [24279, 24280, 24293]}

# dictionary of required columns inside each of the sqlite database tables
df_columns = {}
df_columns['Image'] = ['TableNumber', 'Image_Count_Cells', 'Image_Count_Cytoplasm', 'Image_Count_Nuclei', 'Image_ExecutionTime_01LoadData', 'Image_ExecutionTime_02CorrectIlluminationApply', 'Image_ExecutionTime_03MeasureImageQuality', 'Image_ExecutionTime_04MeasureImageQuality', 'Image_ExecutionTime_06IdentifyPrimaryObjects', 'Image_ExecutionTime_07IdentifySecondaryObjects', 'Image_ExecutionTime_08IdentifyTertiaryObjects', 'Image_ExecutionTime_09MeasureCorrelation', 'Image_ExecutionTime_10MeasureGranularity', 'Image_ExecutionTime_11MeasureObjectIntensity', 'Image_ExecutionTime_12MeasureObjectNeighbors', 'Image_ExecutionTime_13MeasureObjectNeighbors', 'Image_ExecutionTime_14MeasureObjectNeighbors', 'Image_ExecutionTime_15MeasureObjectIntensityDistribution', 'Image_ExecutionTime_16MeasureObjectSizeShape', 'Image_ExecutionTime_17MeasureTexture', 'Image_ExecutionTime_18OverlayOutlines', 'Image_ExecutionTime_19OverlayOutlines', 'Image_ExecutionTime_20SaveImages', 'Image_ExecutionTime_21SaveImages', 'Image_FileName_CellOutlines', 'Image_FileName_IllumAGP', 'Image_FileName_IllumDNA', 'Image_FileName_IllumER', 'Image_FileName_IllumMito', 'Image_FileName_IllumRNA', 'Image_FileName_NucleiOutlines', 'Image_FileName_OrigAGP', 'Image_FileName_OrigDNA', 'Image_FileName_OrigER', 'Image_FileName_OrigMito', 'Image_FileName_OrigRNA']
df_columns['Nuclei'] = ['TableNumber', 'ImageNumber', 'ObjectNumber', 'Nuclei_AreaShape_Area', 'Nuclei_AreaShape_Center_X', 'Nuclei_AreaShape_Center_Y', 'Nuclei_AreaShape_Compactness', 'Nuclei_AreaShape_Eccentricity', 'Nuclei_AreaShape_EulerNumber', 'Nuclei_AreaShape_Extent', 'Nuclei_AreaShape_FormFactor', 'Nuclei_AreaShape_MajorAxisLength', 'Nuclei_AreaShape_MaxFeretDiameter', 'Nuclei_AreaShape_MaximumRadius', 'Nuclei_AreaShape_MeanRadius', 'Nuclei_AreaShape_MedianRadius', 'Nuclei_AreaShape_MinFeretDiameter', 'Nuclei_AreaShape_MinorAxisLength', 'Nuclei_AreaShape_Orientation', 'Nuclei_AreaShape_Perimeter', 'Nuclei_AreaShape_Solidity']
df_columns['Cytoplasm'] = ['TableNumber', 'ImageNumber', 'ObjectNumber', 'Cytoplasm_AreaShape_Area', 'Cytoplasm_AreaShape_Center_X', 'Cytoplasm_AreaShape_Center_Y', 'Cytoplasm_AreaShape_Compactness', 'Cytoplasm_AreaShape_Eccentricity', 'Cytoplasm_AreaShape_EulerNumber', 'Cytoplasm_AreaShape_Extent', 'Cytoplasm_AreaShape_FormFactor', 'Cytoplasm_AreaShape_MajorAxisLength', 'Cytoplasm_AreaShape_MaxFeretDiameter', 'Cytoplasm_AreaShape_MaximumRadius', 'Cytoplasm_AreaShape_MeanRadius', 'Cytoplasm_AreaShape_MedianRadius', 'Cytoplasm_AreaShape_MinFeretDiameter', 'Cytoplasm_AreaShape_MinorAxisLength', 'Cytoplasm_AreaShape_Orientation', 'Cytoplasm_AreaShape_Perimeter', 'Cytoplasm_AreaShape_Solidity']
df_columns['Cells'] = ['TableNumber', 'ImageNumber', 'ObjectNumber', 'Cells_AreaShape_Area', 'Cells_AreaShape_Center_X', 'Cells_AreaShape_Center_Y', 'Cells_AreaShape_Compactness', 'Cells_AreaShape_Eccentricity', 'Cells_AreaShape_EulerNumber', 'Cells_AreaShape_Extent', 'Cells_AreaShape_FormFactor', 'Cells_AreaShape_MajorAxisLength', 'Cells_AreaShape_MaxFeretDiameter', 'Cells_AreaShape_MaximumRadius', 'Cells_AreaShape_MeanRadius', 'Cells_AreaShape_MedianRadius', 'Cells_AreaShape_MinFeretDiameter', 'Cells_AreaShape_MinorAxisLength', 'Cells_AreaShape_Orientation', 'Cells_AreaShape_Perimeter', 'Cells_AreaShape_Solidity']

# create a blank dictionary to save each table as a dataframe
dataframes = {}
# initialize the dictionary with creating each table with specified columns + plateID column
for table in df_columns.keys():
    dataframes[table] = pd.DataFrame(columns=df_columns[table] + ['PlateID'])

for plate in plate_list[analysis_mode]:

    # update path to the sqlite database
    db_path = '../Data/bray2017/'+str(plate)+'/extracted_features/'+str(plate)+'.sqlite'
    # update db connector and cursor
    conn = get_db_conn(db_path=db_path)
    cursor = get_db_cursor(db_path=db_path)

    for table in get_db_tables(cursor):
        # update the query with the next table name and column information
        query = f"SELECT {', '.join(df_columns[table])} FROM {table};"
        # extract the dataframe using query
        temp = pd.read_sql_query(query, conn)

        # add plateID information to newly extracted dataframe
        temp['PlateID'] = plate

        # append the fresh dataframe with the existing dataframe
        dataframes[table] = pd.concat([dataframes[table], temp], ignore_index=True)
        print(f"PlateID: {plate} - Table: {table} | Success")

# add wellID and fieldID information to all dataframes
dataframes = add_id_info(dataframes)

# fix the column orders and data types
dataframes = fix_columns(dataframes)

/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/414771079.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[table] = pd.concat([dataframes[table], temp], ignore_index=True)


PlateID: 24278 - Table: Image | Success


/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/414771079.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[table] = pd.concat([dataframes[table], temp], ignore_index=True)


PlateID: 24278 - Table: Nuclei | Success


/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/414771079.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[table] = pd.concat([dataframes[table], temp], ignore_index=True)


PlateID: 24278 - Table: Cytoplasm | Success


/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/414771079.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[table] = pd.concat([dataframes[table], temp], ignore_index=True)


PlateID: 24278 - Table: Cells | Success
PlateID: 24279 - Table: Image | Success
PlateID: 24279 - Table: Nuclei | Success
PlateID: 24279 - Table: Cytoplasm | Success
PlateID: 24279 - Table: Cells | Success
PlateID: 24280 - Table: Image | Success
PlateID: 24280 - Table: Nuclei | Success
PlateID: 24280 - Table: Cytoplasm | Success
PlateID: 24280 - Table: Cells | Success


/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/2630734150.py:97: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframes[table][eachColumn] = dataframes[table][eachColumn].replace('nan', np.nan)
/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/2630734150.py:97: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframes[table][eachColumn] = dataframes[table][eachColumn].replace('nan', np.nan)


### (Optional) Exporting CSV

In [393]:
# # set up the query parameters
# # table_name = 'Image'
# row_limit = 3

# for table in db_tables:
#     print(f"\n{table}")

#     cursor.execute(f"PRAGMA table_info({table});") # cursor call for retrieving table information
#     columns = cursor.fetchall()
#     column_names = [col[1] for col in columns]
#     print(column_names)

#     query = f"SELECT * FROM {table} LIMIT {row_limit};"  # set up the query using parameters
#     cursor.execute(query) # execute the cursor call using the query
#     rows = cursor.fetchall() # fetch the output of the cursor
#     for row in rows:
#         print(row)

# sampleDataframes = {} # create a blank dictionary to save each table as a dataframe, with a table_name key
# row_limit = 10

# for table in db_tables: # loop through all tables 
    
#     query = f"SELECT * FROM {table} LIMIT {row_limit};"  # set up the query using parameters
#     df = pd.read_sql_query(query, conn) # use the open connector to pull table into

#     sampleDataframes[table] = df # store the sample df in the dictionary
#     print(f"Table {table} has been saved.")

# for table_name, df in sampleDataframes.items():
#     # Export each DataFrame to a CSV file
#     df.to_csv(f"{table_name}.csv", index=False)  # index=False avoids writing row numbers
#     print(f"Exported {table_name} to {table_name}.csv")


In [394]:
# close the connection when the sqlite database processing is done
conn.close()

## Data: Seal et al. (2024)

Ola linked the compounds identified in the BioMorph study with the metadata of the Cell Painting dataset. She linked 603 compounds, which results in 5025 wells spread over 94 different plates.
- CPD_NAME: specific compounds
- Metadata_Plate: specific plates
- Metadata_Well: specific wells

### Endpoint Definitions

1. Apoptosis Up:
- Apoptosis is the process of programmed cell death, where cells die in a controlled manner as part of normal development or in response to damage.
- "**Apoptosis up**" means an increase in the rate of apoptosis in response to a compound, suggesting the compound is inducing cell death via the apoptotic pathway. This could be important in cancer treatments, where the goal is to promote the death of harmful cells.
 2. Cytotoxicity BLA:
- Cytotoxicity refers to the toxic effect a compound has on cells, leading to cell damage or death.
- BLA stands for Beta-Lactamase assay, a biochemical assay often used to detect cytotoxicity. The "**Cytotoxicity BLA**" endpoint indicates cell death or damage measured through the Beta-Lactamase assay.
 3. Cytotoxicity SRB:
- Similar to Cytotoxicity BLA, this measures cell toxicity, but using a different assay.
- SRB stands for Sulforhodamine B, a dye that binds to cellular proteins, and it’s commonly used to measure cell density and viability. The "**Cytotoxicity SRB**" endpoint measures the cytotoxic effect of compounds based on the amount of protein-bound SRB dye, indicating cell death or reduced viability.
 4. ER Stress:
- ER stress refers to stress in the Endoplasmic Reticulum (ER), a cell organelle involved in protein folding and secretion. When misfolded proteins accumulate, ER stress triggers the Unfolded Protein Response (UPR).
- The "ER stress" endpoint indicates that a compound is causing stress in the ER, potentially leading to apoptosis or other cellular dysfunctions.
 5. Heat Shock:
- Heat shock refers to the stress response of cells to elevated temperatures, which results in the production of heat shock proteins (HSPs) that help protect cells from damage.
- The "Heat Shock" endpoint suggests the compound is inducing a cellular response similar to what happens when cells are exposed to heat or other stresses, typically leading to the production of HSPs.
 6. Microtubule Up:
- Microtubules are part of the cell's cytoskeleton and are crucial for cell division and intracellular transport.
- "Microtubule up" indicates an increase in microtubule stabilization or polymerization due to the compound. Compounds that affect microtubules can disrupt cell division, making this endpoint important in cancer research (e.g., chemotherapy drugs like taxanes target microtubules).
 7. Mitochondrial Disruption Up:
- Mitochondria are the energy-producing organelles in cells, and mitochondrial disruption can lead to cell death or dysfunction.
- "Mitochondrial disruption up" indicates an increase in mitochondrial dysfunction, which can lead to cellular energy depletion and apoptosis. This endpoint is used to measure the impact of a compound on mitochondrial health.
 8. Oxidative Stress Up:
- Oxidative stress occurs when there is an imbalance between the production of reactive oxygen species (ROS) and the cell’s ability to detoxify them, leading to cellular damage.
- "Oxidative stress up" means the compound is causing an increase in oxidative stress, which can damage DNA, proteins, and lipids, potentially leading to cell death.
 9. Proliferation Decrease:
- Proliferation refers to the growth and division of cells. A decrease in proliferation means that the cells are dividing more slowly or not at all.
- "Proliferation decrease" indicates that the compound is inhibiting cell growth. This endpoint is often used in cancer research to evaluate the efficacy of treatments designed to slow or stop the growth of tumor cells.

### Loading & Cleaning

In [395]:
# read the xlsx file
biomorph = pd.read_excel("../Data/olaBiomorph/603_compounds_metadata.xlsx")

# view a snippet of the original dataset
# biomorph.head()

# choose the important columns
biomorph_columns = ['Metadata_Plate',	
                    'Metadata_Well',	
                    'CPD_NAME',	'CPD_SAMPLE_ID',	
                    'apoptosis up',	
                    'cytotoxicity BLA',
                    'cytotoxicity SRB',	
                    'ER stress',	
                    'heat shock',	
                    'microtubule up',	
                    'mitochondrial disruption up',	
                    'oxidative stress up', 
                    'proliferation decrease']

# list of toxicity endpoints
endpoint_columns = ['apoptosis up', 'cytotoxicity BLA', 'cytotoxicity SRB', 'ER stress', 
                    'heat shock', 'microtubule up', 'mitochondrial disruption up', 
                    'oxidative stress up', 'proliferation decrease']

# remove the unnecessary columns
biomorph = biomorph.loc[:, biomorph_columns]

# improve the column names
biomorph.rename(columns={'Metadata_Plate': 'PlateID'}, inplace=True)
biomorph.rename(columns={'Metadata_Well': 'WellID'}, inplace=True)

# remove the unused plate information
biomorph = biomorph[biomorph.PlateID.isin(plate_list[analysis_mode])]

# create a column with total endpoint sum
biomorph['total_endpoints'] = biomorph[endpoint_columns].sum(axis=1)

# add a column with a tuple list of total endpoint activities
biomorph['endpoint_combination'] = biomorph[endpoint_columns].apply(tuple, axis=1)

# quick inspection
print(biomorph.shape)
print(biomorph.info())

# print the snippet of the cleaned dataset
biomorph

(297, 15)
<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 103 to 399
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   PlateID                      297 non-null    int64 
 1   WellID                       297 non-null    object
 2   CPD_NAME                     297 non-null    object
 3   CPD_SAMPLE_ID                297 non-null    object
 4   apoptosis up                 297 non-null    int64 
 5   cytotoxicity BLA             297 non-null    int64 
 6   cytotoxicity SRB             297 non-null    int64 
 7   ER stress                    297 non-null    int64 
 8   heat shock                   297 non-null    int64 
 9   microtubule up               297 non-null    int64 
 10  mitochondrial disruption up  297 non-null    int64 
 11  oxidative stress up          297 non-null    int64 
 12  proliferation decrease       297 non-null    int64 
 13  total_endpoints             

,PlateID,WellID,CPD_NAME,CPD_SAMPLE_ID,apoptosis up,cytotoxicity BLA,cytotoxicity SRB,ER stress,heat shock,microtubule up,mitochondrial disruption up,oxidative stress up,proliferation decrease,total_endpoints,endpoint_combination
103,24278,a03,olmesartan medoxomil,SA59556,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
104,24278,a06,citropten,SA59278,0,0,0,0,0,0,1,0,0,1,"(0, 0, 0, 0, 0, 0, 1, 0, 0)"
105,24278,a09,bromperidol,SA83338,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
106,24278,a10,leflunomide,SA792771,1,1,0,1,1,0,0,1,0,5,"(1, 1, 0, 1, 1, 0, 0, 1, 0)"
107,24278,a11,suxibuzone,SA58544,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,24280,p13,chlorpropamide,SA58227,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
396,24280,p14,nicorandil,SA83824,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
397,24280,p17,prednisolone,SA83046,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
398,24280,p21,chlorzoxazone,SA82767,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"


# 2. Explatory Data Analysis

## Table Snippets

In [396]:
print(dataframes['Image'].info())
dataframes['Image'].head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6853 entries, 0 to 6852
Data columns (total 39 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   TableID                                                   6853 non-null   object 
 1   PlateID                                                   6853 non-null   object 
 2   WellID                                                    6853 non-null   object 
 3   FieldID                                                   6853 non-null   object 
 4   Image_Count_Cells                                         6853 non-null   float64
 5   Image_Count_Cytoplasm                                     6853 non-null   float64
 6   Image_Count_Nuclei                                        6853 non-null   float64
 7   Image_ExecutionTime_01LoadData                            6853 non-null   float64
 8   Image_ExecutionTim

,TableID,PlateID,WellID,FieldID,Image_Count_Cells,Image_Count_Cytoplasm,Image_Count_Nuclei,Image_ExecutionTime_01LoadData,Image_ExecutionTime_02CorrectIlluminationApply,Image_ExecutionTime_03MeasureImageQuality,...,Image_FileName_IllumDNA,Image_FileName_IllumER,Image_FileName_IllumMito,Image_FileName_IllumRNA,Image_FileName_NucleiOutlines,Image_FileName_OrigAGP,Image_FileName_OrigDNA,Image_FileName_OrigER,Image_FileName_OrigMito,Image_FileName_OrigRNA
0,0702991209138712afb02ac7ea637f71,24278,a01,s1,58.0,58.0,58.0,4.33,0.05,6.79,...,24278_IllumDNA.mat,24278_IllumER.mat,24278_IllumMito.mat,24278_IllumRNA.mat,a01_s1--nuclei_outlines.png,cdp2bioactives_a01_s1_w46d2c0547-8e3b-440e-a85...,cdp2bioactives_a01_s1_w1bfb15712-b306-40fd-a77...,cdp2bioactives_a01_s1_w2edcec6dc-b1e3-4ffc-80d...,cdp2bioactives_a01_s1_w5d4e4b98c-0f39-4db9-91b...,cdp2bioactives_a01_s1_w336f7b0bc-6ae8-4667-a6a...
1,e2238b50acc3114c310acbf4c68bd114,24278,a01,s2,35.0,35.0,35.0,4.29,0.03,5.95,...,24278_IllumDNA.mat,24278_IllumER.mat,24278_IllumMito.mat,24278_IllumRNA.mat,a01_s2--nuclei_outlines.png,cdp2bioactives_a01_s2_w4da2481a5-d23f-4aae-b35...,cdp2bioactives_a01_s2_w1bd0b9bc7-0d8b-48ed-b04...,cdp2bioactives_a01_s2_w2abeb5a62-b570-447f-97f...,cdp2bioactives_a01_s2_w57540fec0-d693-46a8-bd6...,cdp2bioactives_a01_s2_w3b3fb2060-153d-4096-af5...
2,b983ac6d8cc9a5ed9b713585e32ac4ae,24278,a01,s3,26.0,26.0,26.0,4.35,0.03,5.30,...,24278_IllumDNA.mat,24278_IllumER.mat,24278_IllumMito.mat,24278_IllumRNA.mat,a01_s3--nuclei_outlines.png,cdp2bioactives_a01_s3_w48ffcfde9-24ae-486c-844...,cdp2bioactives_a01_s3_w17290b03d-9255-40d6-898...,cdp2bioactives_a01_s3_w268c116dd-b84b-4a22-94d...,cdp2bioactives_a01_s3_w5b3744705-89fc-4e6e-9c7...,cdp2bioactives_a01_s3_w3334a07f0-46b6-4e3d-8d8...
3,726ad7ac7c4813097cc3aab610c143b4,24278,a01,s4,55.0,55.0,55.0,4.70,0.04,6.84,...,24278_IllumDNA.mat,24278_IllumER.mat,24278_IllumMito.mat,24278_IllumRNA.mat,a01_s4--nuclei_outlines.png,cdp2bioactives_a01_s4_w46dbd1f1e-f3cd-4590-ba2...,cdp2bioactives_a01_s4_w1d07860e9-2432-4233-96b...,cdp2bioactives_a01_s4_w2a8a9c55a-b3b7-4f4d-b70...,cdp2bioactives_a01_s4_w5752fd3e8-0997-4cf1-8a8...,cdp2bioactives_a01_s4_w3e9947106-9c5d-428a-b8a...
4,9a0fed6eaf4ee63b89adc60e02bfbb58,24278,a01,s5,21.0,21.0,21.0,4.25,0.03,6.44,...,24278_IllumDNA.mat,24278_IllumER.mat,24278_IllumMito.mat,24278_IllumRNA.mat,a01_s5--nuclei_outlines.png,cdp2bioactives_a01_s5_w41bba91bb-81ff-4862-af3...,cdp2bioactives_a01_s5_w1ac6937d8-cbbe-40d4-899...,cdp2bioactives_a01_s5_w20f38032f-7e71-41cc-94c...,cdp2bioactives_a01_s5_w518bc5494-c49b-4c7f-a94...,cdp2bioactives_a01_s5_w373e8a6f0-ac34-4be4-8b1...


In [397]:
print(dataframes['Cells'].info())
dataframes['Cells'].head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701759 entries, 0 to 701758
Data columns (total 24 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   TableID                           701759 non-null  object 
 1   PlateID                           701759 non-null  object 
 2   WellID                            701759 non-null  object 
 3   FieldID                           701759 non-null  object 
 4   ImageNumber                       701759 non-null  object 
 5   ObjectNumber                      701759 non-null  object 
 6   Cells_AreaShape_Area              701759 non-null  int64  
 7   Cells_AreaShape_Center_X          701759 non-null  float64
 8   Cells_AreaShape_Center_Y          701759 non-null  float64
 9   Cells_AreaShape_Compactness       701730 non-null  float64
 10  Cells_AreaShape_Eccentricity      701730 non-null  float64
 11  Cells_AreaShape_EulerNumber       701759 non-null  f

,TableID,PlateID,WellID,FieldID,ImageNumber,ObjectNumber,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,...,Cells_AreaShape_MajorAxisLength,Cells_AreaShape_MaxFeretDiameter,Cells_AreaShape_MaximumRadius,Cells_AreaShape_MeanRadius,Cells_AreaShape_MedianRadius,Cells_AreaShape_MinFeretDiameter,Cells_AreaShape_MinorAxisLength,Cells_AreaShape_Orientation,Cells_AreaShape_Perimeter,Cells_AreaShape_Solidity
0,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,1,1634,43.0,0.0,1.509013,...,70.397234,80.709355,21.023796,7.075857,6.082763,40.703919,36.222035,63.738571,230.324,0.836447
1,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,2,2999,619.0,28.0,1.487299,...,97.565456,112.507778,20.000000,7.415620,6.708204,51.680329,42.534725,-34.521891,327.642,0.796441
2,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,3,2471,219.0,33.0,1.126171,...,68.268046,70.710678,20.615528,7.682926,7.000000,51.342921,49.118568,-73.795204,243.636,0.853099
3,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,4,1951,265.0,32.0,1.383061,...,75.198694,81.492331,17.691806,6.372729,5.830952,37.103120,34.684289,-28.019719,217.738,0.913390
4,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,5,1619,82.0,11.0,1.119820,...,54.504681,58.591808,18.681542,7.630574,7.000000,41.751358,40.508727,-70.130892,187.254,0.893241


In [398]:
print(dataframes['Nuclei'].info())
dataframes['Nuclei'].head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701759 entries, 0 to 701758
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   TableID                            701759 non-null  object 
 1   PlateID                            701759 non-null  object 
 2   WellID                             701759 non-null  object 
 3   FieldID                            701759 non-null  object 
 4   ImageNumber                        701759 non-null  object 
 5   ObjectNumber                       701759 non-null  object 
 6   Nuclei_AreaShape_Area              701759 non-null  int64  
 7   Nuclei_AreaShape_Center_X          701759 non-null  float64
 8   Nuclei_AreaShape_Center_Y          701759 non-null  float64
 9   Nuclei_AreaShape_Compactness       701759 non-null  float64
 10  Nuclei_AreaShape_Eccentricity      701759 non-null  float64
 11  Nuclei_AreaShape_EulerNumber       7017

,TableID,PlateID,WellID,FieldID,ImageNumber,ObjectNumber,Nuclei_AreaShape_Area,Nuclei_AreaShape_Center_X,Nuclei_AreaShape_Center_Y,Nuclei_AreaShape_Compactness,...,Nuclei_AreaShape_MajorAxisLength,Nuclei_AreaShape_MaxFeretDiameter,Nuclei_AreaShape_MaximumRadius,Nuclei_AreaShape_MeanRadius,Nuclei_AreaShape_MedianRadius,Nuclei_AreaShape_MinFeretDiameter,Nuclei_AreaShape_MinorAxisLength,Nuclei_AreaShape_Orientation,Nuclei_AreaShape_Perimeter,Nuclei_AreaShape_Solidity
0,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,1,668,52.0,20.0,1.036701,...,33.056429,33.600595,13.000000,4.922154,4.242641,25.491175,25.944878,50.159928,100.076,0.948864
1,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,2,890,640.0,23.0,1.261758,...,47.799265,46.615448,12.041595,5.001021,4.472136,23.574758,23.961891,-34.081184,122.700,0.957504
2,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,3,917,220.0,25.0,1.183880,...,45.106195,42.059482,12.165525,5.043240,4.472136,27.496545,27.015068,-34.750027,127.356,0.921145
3,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,4,699,265.0,28.0,1.028651,...,33.465476,33.600595,13.416408,5.127288,4.472136,26.162951,26.708074,-42.291306,98.904,0.958848
4,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,5,862,79.0,30.0,1.235444,...,45.684766,43.680659,12.369317,4.861480,4.472136,26.667468,24.993011,73.200746,126.114,0.909283


In [399]:
print(dataframes['Cytoplasm'].info())
dataframes['Cytoplasm'].head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701759 entries, 0 to 701758
Data columns (total 24 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   TableID                               701759 non-null  object 
 1   PlateID                               701759 non-null  object 
 2   WellID                                701759 non-null  object 
 3   FieldID                               701759 non-null  object 
 4   ImageNumber                           701759 non-null  object 
 5   ObjectNumber                          701759 non-null  object 
 6   Cytoplasm_AreaShape_Area              701759 non-null  int64  
 7   Cytoplasm_AreaShape_Center_X          701759 non-null  float64
 8   Cytoplasm_AreaShape_Center_Y          701759 non-null  float64
 9   Cytoplasm_AreaShape_Compactness       701730 non-null  float64
 10  Cytoplasm_AreaShape_Eccentricity      701730 non-null  float64
 11  

,TableID,PlateID,WellID,FieldID,ImageNumber,ObjectNumber,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,...,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MaxFeretDiameter,Cytoplasm_AreaShape_MaximumRadius,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_MedianRadius,Cytoplasm_AreaShape_MinFeretDiameter,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity
0,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,1,1082,34.0,0.0,3.082407,...,83.722955,80.709355,14.142136,3.867136,3.000000,40.703919,38.264106,64.718032,323.744,0.553878
1,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,2,2257,608.0,43.0,2.154230,...,101.018066,112.507778,13.892444,4.497375,3.605551,51.680329,46.289561,-37.449278,443.272,0.599389
2,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,3,1707,221.0,52.0,1.911851,...,77.918782,70.710678,13.453624,4.171019,3.605551,51.342921,47.131865,-73.961135,362.336,0.589332
3,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,4,1368,290.0,24.0,2.533414,...,87.519053,81.492331,10.630146,3.582114,3.000000,37.103120,33.834025,-27.396395,310.986,0.640449
4,0702991209138712afb02ac7ea637f71,24278,a01,s1,1,5,905,87.0,0.0,2.404373,...,60.685292,58.591808,12.000000,3.816261,3.000000,41.751358,43.015571,-67.190934,303.642,0.499310


## Cell Counts

In [400]:
# calculate the cells counts of each plate, each well and each field
cells_per_plate = dataframes['Cells'].groupby('PlateID').size().reset_index(name='Number_of_Cells')
cells_per_well = dataframes['Cells'].groupby(['PlateID', 'WellID']).size().reset_index(name='Number_of_Cells')
cells_per_field = dataframes['Cells'].groupby(['PlateID', 'WellID', 'FieldID']).size().reset_index(name='Number_of_Cells')

biomorph = biomorph.merge(right=cells_per_well[['PlateID', 'WellID', 'Number_of_Cells']],
                          on=['PlateID', 'WellID'],
                          how='left')

biomorph = move_column(df=biomorph, col_name='Number_of_Cells', new_position=2)
biomorph

,PlateID,WellID,Number_of_Cells,CPD_NAME,CPD_SAMPLE_ID,apoptosis up,cytotoxicity BLA,cytotoxicity SRB,ER stress,heat shock,microtubule up,mitochondrial disruption up,oxidative stress up,proliferation decrease,total_endpoints,endpoint_combination
0,24278,a03,434,olmesartan medoxomil,SA59556,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
1,24278,a06,372,citropten,SA59278,0,0,0,0,0,0,1,0,0,1,"(0, 0, 0, 0, 0, 0, 1, 0, 0)"
2,24278,a09,418,bromperidol,SA83338,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
3,24278,a10,358,leflunomide,SA792771,1,1,0,1,1,0,0,1,0,5,"(1, 1, 0, 1, 1, 0, 0, 1, 0)"
4,24278,a11,384,suxibuzone,SA58544,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,24280,p13,692,chlorpropamide,SA58227,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
293,24280,p14,712,nicorandil,SA83824,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
294,24280,p17,727,prednisolone,SA83046,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"
295,24280,p21,694,chlorzoxazone,SA82767,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)"


In [401]:
# # Cell count distribution of wells
# cells_per_well.hist(column=['Number_of_Cells'], bins=25, grid=True, alpha=0.6, color='skyblue')
# plt.title('Number of Cells Per Well')
# plt.xlabel('Cell Count')
# plt.ylabel('# of Wells')
# plt.tight_layout()
# plt.show()

## Well Statistics

Aggregating FOVs of each well, compute the mean and standard deviation of cells and nuclei area:

In [402]:
# calculate the aggregated cell and nuclei metrics per well
cell_agg_per_well = dataframes['Cells'].groupby('WellID').agg(Cells_AreaShape_Area_Mean=('Cells_AreaShape_Area', 'mean'),
                                               Cells_AreaShape_Area_Std=('Cells_AreaShape_Area', 'std'),
                                               Cells_AreaShape_Compactness_Mean=('Cells_AreaShape_Compactness', 'mean'),
                                               Cells_AreaShape_Compactness_Std=('Cells_AreaShape_Compactness', 'std')).reset_index()

nuclei_agg_per_well = dataframes['Nuclei'].groupby('WellID').agg(Nuclei_AreaShape_Area_Mean=('Nuclei_AreaShape_Area', 'mean'),
                                                                      Nuclei_AreaShape_Area_Std=('Nuclei_AreaShape_Area', 'std'),
                                                                      Nuclei_AreaShape_Compactness_Mean=('Nuclei_AreaShape_Compactness', 'mean'),
                                                                      Nuclei_AreaShape_Compactness_Std=('Nuclei_AreaShape_Compactness', 'std')).reset_index()

# merge aggregated metrics with the biomorph endpoints table
biomorph_well_stats = biomorph.merge(cell_agg_per_well, on='WellID', how='left')
biomorph_well_stats = biomorph_well_stats.merge(nuclei_agg_per_well, on='WellID', how='left')
biomorph_well_stats

,PlateID,WellID,Number_of_Cells,CPD_NAME,CPD_SAMPLE_ID,apoptosis up,cytotoxicity BLA,cytotoxicity SRB,ER stress,heat shock,...,total_endpoints,endpoint_combination,Cells_AreaShape_Area_Mean,Cells_AreaShape_Area_Std,Cells_AreaShape_Compactness_Mean,Cells_AreaShape_Compactness_Std,Nuclei_AreaShape_Area_Mean,Nuclei_AreaShape_Area_Std,Nuclei_AreaShape_Compactness_Mean,Nuclei_AreaShape_Compactness_Std
0,24278,a03,434,olmesartan medoxomil,SA59556,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2687.518144,1454.575949,1.388319,0.327881,782.198096,293.822368,1.130164,0.114210
1,24278,a06,372,citropten,SA59278,0,0,0,0,0,...,1,"(0, 0, 0, 0, 0, 0, 1, 0, 0)",2962.972316,1428.201699,1.372494,0.318200,813.108035,317.353138,1.121929,0.103381
2,24278,a09,418,bromperidol,SA83338,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2932.884336,1511.599978,1.343568,0.287049,849.709848,348.472678,1.120769,0.106493
3,24278,a10,358,leflunomide,SA792771,1,1,0,1,1,...,5,"(1, 1, 0, 1, 1, 0, 0, 1, 0)",2857.346030,1713.853495,1.311371,0.304601,953.353777,704.032332,1.119216,0.125520
4,24278,a11,384,suxibuzone,SA58544,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2645.699280,1366.797826,1.338747,0.291638,822.266507,359.961649,1.128288,0.118696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,24280,p13,692,chlorpropamide,SA58227,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2775.379716,1584.417681,1.299271,0.247719,849.069264,340.247699,1.114420,0.106113
293,24280,p14,712,nicorandil,SA83824,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2611.194021,1345.938476,1.309494,0.245088,836.831923,312.709051,1.119915,0.104672
294,24280,p17,727,prednisolone,SA83046,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2606.310929,1293.146337,1.340983,0.282173,821.093989,284.626353,1.126077,0.107726
295,24280,p21,694,chlorzoxazone,SA82767,0,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2745.614980,1522.006436,1.293684,0.268221,872.931539,339.809022,1.110063,0.107985


## Field Statistics

Because the approach above resulted in less-than-ideal number of instances, let's aggregate for each FOV instead of each well:

In [403]:
# populate the endpoint dataframe with all available fields of each well
biomorph_field_stats = biomorph.merge(dataframes['Cells'][['PlateID', 'WellID', 'FieldID']].drop_duplicates(), 
                     on=['PlateID', 'WellID'], 
                     how='left')
biomorph_field_stats = move_column(df=biomorph_field_stats, col_name='FieldID', new_position=2)

# calculate mean and standard deviation for 'Cells_AreaShape_Area' and 'Cells_AreaShape_Compactness' in cells_df by FieldID
cell_agg_per_field = dataframes['Cells'].groupby(['PlateID', 'WellID', 'FieldID']).agg(
    Cells_AreaShape_Area_Mean=('Cells_AreaShape_Area', 'mean'),
    Cells_AreaShape_Area_Std=('Cells_AreaShape_Area', 'std'),
    Cells_AreaShape_Compactness_Mean=('Cells_AreaShape_Compactness', 'mean'),
    Cells_AreaShape_Compactness_Std=('Cells_AreaShape_Compactness', 'std')
).reset_index()

# merge the aggregated metrics from cells_df into endpoint dataframe
biomorph_field_stats = biomorph_field_stats.merge(cell_agg_per_field, on=['PlateID', 'WellID', 'FieldID'], how='left')

# calculate mean and standard deviation for 'Cells_AreaShape_Area' and 'Cells_AreaShape_Compactness' in nuclei_df by FieldID
nuclei_agg_per_field = dataframes['Nuclei'].groupby(['PlateID', 'WellID', 'FieldID']).agg(
    Nuclei_AreaShape_Area_Mean=('Nuclei_AreaShape_Area', 'mean'),
    Nuclei_AreaShape_Area_Std=('Nuclei_AreaShape_Area', 'std'),
    Nuclei_AreaShape_Compactness_Mean=('Nuclei_AreaShape_Compactness', 'mean'),
    Nuclei_AreaShape_Compactness_Std=('Nuclei_AreaShape_Compactness', 'std')).reset_index()

# merge the aggregated metrics from nuclei_df into endpoint dataframe
biomorph_field_stats = biomorph_field_stats.merge(nuclei_agg_per_field, on=['PlateID', 'WellID', 'FieldID'], how='left')

# add the cell counts for each field of each well
biomorph_field_stats = biomorph_field_stats.drop('Number_of_Cells', axis=1)
biomorph_field_stats = biomorph_field_stats.merge(cells_per_field, on=['PlateID', 'WellID', 'FieldID'], how='left')
biomorph_field_stats = move_column(df=biomorph_field_stats, col_name='Number_of_Cells', new_position=3)
biomorph_field_stats

,PlateID,WellID,FieldID,Number_of_Cells,CPD_NAME,CPD_SAMPLE_ID,apoptosis up,cytotoxicity BLA,cytotoxicity SRB,ER stress,...,total_endpoints,endpoint_combination,Cells_AreaShape_Area_Mean,Cells_AreaShape_Area_Std,Cells_AreaShape_Compactness_Mean,Cells_AreaShape_Compactness_Std,Nuclei_AreaShape_Area_Mean,Nuclei_AreaShape_Area_Std,Nuclei_AreaShape_Compactness_Mean,Nuclei_AreaShape_Compactness_Std
0,24278,a03,s1,97,olmesartan medoxomil,SA59556,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2697.989691,2184.240555,1.362183,0.320399,801.216495,353.681714,1.122623,0.097678
1,24278,a03,s2,110,olmesartan medoxomil,SA59556,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",2446.090909,1270.143419,1.369504,0.283888,798.418182,348.722502,1.133376,0.111364
2,24278,a03,s3,49,olmesartan medoxomil,SA59556,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3826.408163,2494.539377,1.454695,0.437254,854.448980,374.762095,1.136081,0.106685
3,24278,a03,s4,74,olmesartan medoxomil,SA59556,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3483.702703,2104.109617,1.546692,0.533525,821.567568,385.582643,1.148037,0.172811
4,24278,a03,s5,61,olmesartan medoxomil,SA59556,0,0,0,0,...,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3414.852459,1483.216065,1.573668,0.464460,739.688525,224.409042,1.110631,0.080053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,24280,p24,s2,97,desoxycortone,SA83656,0,0,0,0,...,2,"(0, 0, 0, 0, 1, 0, 0, 1, 0)",2773.185567,1377.933872,1.315346,0.200231,814.103093,321.405652,1.124542,0.100566
1770,24280,p24,s3,133,desoxycortone,SA83656,0,0,0,0,...,2,"(0, 0, 0, 0, 1, 0, 0, 1, 0)",2013.368421,908.603094,1.320185,0.260929,731.609023,276.919632,1.141539,0.134286
1771,24280,p24,s4,56,desoxycortone,SA83656,0,0,0,0,...,2,"(0, 0, 0, 0, 1, 0, 0, 1, 0)",3807.821429,1445.826949,1.429327,0.344007,773.160714,267.432899,1.134623,0.102193
1772,24280,p24,s5,84,desoxycortone,SA83656,0,0,0,0,...,2,"(0, 0, 0, 0, 1, 0, 0, 1, 0)",3021.488095,2437.344746,1.383048,0.356189,896.250000,728.147193,1.138016,0.111861


## Compound & Endpoint Combination

In [404]:
print(f"The number of samples with no endpoint activity: {biomorph[biomorph['total_endpoints'] == 0].shape[0]}")
print(f"The number of samples with a singular endpoint activity: {biomorph[biomorph['total_endpoints'] == 1].shape[0]}")
print(f"The number of samples with a multiple endpoint activities: {biomorph[biomorph['total_endpoints'] > 1].shape[0]}")

The number of samples with no endpoint activity: 165
The number of samples with a singular endpoint activity: 60
The number of samples with a multiple endpoint activities: 72


In [405]:
# retrieve the occurrence counts of endpoint activities
endpoint_combination_counts = biomorph['endpoint_combination'].value_counts()

# group by endpoint combination and then count unique compounds in each group
endpoint_compound_counts = biomorph.groupby('endpoint_combination')['CPD_NAME'].nunique()
endpoint_compound_counts = endpoint_compound_counts.sort_values(ascending=False)

print(f'Endpoint Combination Counts:\n{endpoint_combination_counts}\n')
print(f'Endpoint Compound Counts:\n{endpoint_compound_counts}')

Endpoint Combination Counts:
endpoint_combination
(0, 0, 0, 0, 0, 0, 0, 0, 0)    165
(0, 1, 0, 0, 0, 0, 0, 0, 0)     21
(1, 0, 0, 0, 0, 0, 0, 0, 0)     21
(1, 0, 1, 1, 0, 0, 0, 1, 1)      6
(0, 0, 0, 0, 0, 0, 1, 0, 0)      6
(0, 0, 0, 0, 0, 0, 0, 1, 0)      6
(1, 1, 0, 1, 1, 0, 0, 0, 0)      6
(1, 1, 1, 1, 1, 0, 1, 1, 1)      6
(1, 0, 0, 0, 1, 0, 0, 0, 0)      6
(1, 1, 0, 0, 0, 0, 0, 0, 0)      3
(1, 1, 1, 1, 1, 0, 0, 1, 1)      3
(1, 0, 1, 0, 0, 0, 1, 1, 1)      3
(0, 0, 0, 1, 1, 0, 0, 1, 0)      3
(1, 0, 0, 1, 0, 0, 0, 0, 0)      3
(0, 0, 0, 0, 0, 0, 0, 0, 1)      3
(0, 0, 1, 0, 0, 0, 1, 1, 1)      3
(0, 0, 0, 1, 0, 0, 0, 0, 0)      3
(1, 1, 0, 1, 0, 0, 1, 1, 1)      3
(1, 0, 0, 0, 0, 0, 1, 1, 1)      3
(1, 1, 0, 0, 1, 0, 0, 0, 0)      3
(0, 1, 0, 1, 1, 0, 0, 0, 0)      3
(1, 0, 0, 1, 1, 0, 0, 0, 0)      3
(0, 0, 0, 0, 0, 0, 1, 0, 1)      3
(1, 0, 1, 1, 1, 1, 1, 1, 1)      3
(1, 0, 0, 0, 0, 1, 0, 1, 1)      3
(1, 1, 0, 1, 1, 0, 0, 1, 0)      3
(0, 0, 0, 0, 1, 0, 0, 1, 0)      3
Name:

In [406]:
# all of the compounds only results in a single endpoint combination
for eachCompound in biomorph['CPD_NAME'].unique():
    if len(biomorph[biomorph['CPD_NAME'] == eachCompound]['endpoint_combination'].unique()) != 1:
        print(f"{eachCompound} has multiple occurring endpoint combinations")

In [407]:
# calculate total occurrences of each compound
compound_counts = biomorph['CPD_NAME'].value_counts().reset_index()
compound_counts.columns = ['CPD_NAME', 'Total_Occurrences']

# find the most frequent endpoint combination for each compound
most_common_combinations = biomorph.groupby(['CPD_NAME', 'endpoint_combination']).size().reset_index(name='Count')
most_common_combinations = most_common_combinations.sort_values(['CPD_NAME', 'Count'], ascending=[True, False])

# get the top combination for each compound
most_common_combinations = most_common_combinations.drop_duplicates(subset='CPD_NAME', keep='first')

# merge with total occurrences and calculate decimal percentage
compound_summary = pd.merge(compound_counts, most_common_combinations, on='CPD_NAME')
compound_summary['Decimal_Percentage'] = compound_summary['Count'] / compound_summary['Total_Occurrences']

# rename columns for clarity
compound_summary.columns = ['CPD_NAME', 'Total_Occurrences', 'Most_Common_Combination', 'Occurrence_with_Combination', 'Decimal_Percentage']

# sort by total occurrences in descending order
compound_summary = compound_summary.sort_values(by='Total_Occurrences', ascending=False).reset_index(drop=True)
compound_summary

,CPD_NAME,Total_Occurrences,Most_Common_Combination,Occurrence_with_Combination,Decimal_Percentage
0,olmesartan medoxomil,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
1,etofylline,3,"(1, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
2,bromperidol,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
3,leflunomide,3,"(1, 1, 0, 1, 1, 0, 0, 1, 0)",3,1.0
4,suxibuzone,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
...,...,...,...,...,...
94,chlorpropamide,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
95,nicorandil,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
96,prednisolone,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0
97,chlorzoxazone,3,"(0, 0, 0, 0, 0, 0, 0, 0, 0)",3,1.0


## Statistical Testing

**What's our problem?**
- We need to understand whether **every site for a well** is as representative for the *end-points* as the next, or if there’s a good deal of heterogeneity if we were to consider the data on a site-level vs a well level.
- Therefore, the goal is to measure **heterogeneity across fields** within each well.
- If there is high variability across fields, it suggests that a single well-level average may not capture all the underlying variability, which could justify a more granular analysis at the site level.

**How to proceed?**
1. **Test for normality:** Assess if each feature within each site follows a normal distribution, which impacts the choice of parametric vs. non-parametric tests.
    - Shapiro-Wilk test for univariate, Henze-Zirkler's test for multivariate normality if using multiple features
    - Count the number of features that pass/fail normality across fields and wells to evaluate if parametric assumptions hold.
2. **Statistical Testing for Differences Across Sites:** Determine if the observed variability across sites within each well is statistically significant by proceeding with a parametric or non-parametric tests based on normality results.
	- **MANOVA (parametric, for multivariate data)** if normality holds across fields within a well.
	- **PERMANOVA (non-parametric, for multivariate data)** if normality does not hold.
	- **ANOVA or Kruskal-Wallis** for univariate data (for each feature separately).

**What are the options?**
1. I can use **ANOVA (Analysis of Variance)** test to see whether there are statistically significant differences in the means of a variable (in this case, cell metrics like area) across multiple groups (fields within each well).
    - I could have compared the cell-feature means of each field inside a well, but then I'd have very few number of instances (at most 6) for statistical tests.
    - That's why I directly used single-cell metrics instead of field-level aggregates for ANOVA, which helps to improve the statistical power of the test.
    - ***Adding endpoint/compound into the test***: Since the endpoint combination is the same across all fields in each well, it won’t vary within the well. This makes it less relevant for ANOVA, as I'm not *comparing across different endpoint labels* but rather *testing consistency within the fields assigned the same endpoint*.
        - If, however, **significant heterogeneity within fields** for wells labeled with the same endpoint combination is observed, this would indicate that some fields may not represent that endpoint as consistently as others. In such cases, I should follow up by **examining whether certain endpoint combinations show more heterogeneity** across fields than others, which could highlight specific conditions that are less consistent across fields.
    - **Interpretation**: Low p-value (< 0.05) indicates statistically significant differences in a feature between fields within a well. This suggests that field-level heterogeneity exists, supporting the need for site- or single-cell level analysis. High p-value suggests that field-level averages are more similar, implying less heterogeneity across fields within the well.

### Normality Testing

Although **MANOVA** has **multivariate normality** requirement, which means that the distribution of each feature and all combinations of features should be normally distributed within each group (in your case, within each field in a well). This assumption ensures accurate results in MANOVA, as it affects the validity of p-values.

This requirement can be tested by:
- **Univariate Normality: Shapiro-Wilk Test** test each feature individually within each field. It tests normality, but don’t capture interactions between features.

- **Multivariate Normality: Henze-Zirkler test** is more suited for testing multivariate normality across features simultaneously, making it ideal before applying MANOVA. It has a good overall power against alternatives to normality and works for any dimension and sample size.

#### Univariate Normality: Shapiro-Wilk

- The Shapiro-Wilk test can be applied to each field’s feature data within each well to check for **univariate normality** on a per-feature basis.
- The p-value indicates the likelihood of observing the given data if it were normally distributed. 
    - **High p-value** indicates that there's no strong evidence to reject the null hypothesis that the data follows a normal distribution.
    - **Low p-value** causes the null hypothesis of normality to be rejected, suggesting that the data deviates significantly from a normal distribution.

In [546]:
# List to store Shapiro-Wilk test results
shapiro_results = []
lap_counter = 0

# Iterate over each plate and well combination
for plate_id, well_id in dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates().values:

    well_start_time = time.time()
    lap_counter += 1
    
    # subset the respective cell and nuclei data
    well_data_cells = dataframes['Cells'][(dataframes['Cells']['PlateID'] == plate_id) & 
                                          (dataframes['Cells']['WellID'] == well_id)]
    well_data_nuclei = dataframes['Nuclei'][(dataframes['Nuclei']['PlateID'] == plate_id) & 
                                            (dataframes['Nuclei']['WellID'] == well_id)]
    
    # merge cell and nuclei data with an outer join
    well_data = well_data_cells.merge(well_data_nuclei, on=['TableID', 'PlateID', 'WellID', 'FieldID', 'ImageNumber', 'ObjectNumber'], how='outer')
    
    # define the features to include in MANOVA and Mardia's test
    features = ['Cells_AreaShape_Area', 'Cells_AreaShape_Compactness', 'Nuclei_AreaShape_Area', 'Nuclei_AreaShape_Compactness']
    
    # Iterate over each field within this well
    for field_id in well_data['FieldID'].unique():
        
        # Subset the feature data for the current field
        field_data = well_data[well_data['FieldID'] == field_id][['Cells_AreaShape_Area',
                                                                  'Cells_AreaShape_Compactness',
                                                                  'Nuclei_AreaShape_Area',
                                                                  'Nuclei_AreaShape_Compactness']].dropna()
        
        # only perform shapiro-wilk test if there is enough data
        if len(field_data) > 3:
            for feature in field_data.columns:

                # perform shapiro-wilk test for the current feature
                stat, p_value = shapiro(field_data[feature])
                
                # append the results together
                shapiro_results.append({
                    'PlateID': plate_id,
                    'WellID': well_id,
                    'FieldID': field_id,
                    'Feature': feature,
                    'Shapiro_stat': stat,
                    'Shapiro_p_value': p_value,
                    'Normality': p_value > 0.05  # True if the feature data is normal
                })
    
    well_end_time = time.time() - well_start_time
    print(f"Plate: {plate_id}, Well: {well_id}, ETC: {(well_end_time * (len(dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates())
                                                                         - lap_counter)) / 60:.0f}m")

# convert results to df
shapiro_results_df = pd.DataFrame(shapiro_results)
print("Shapiro-Wilk Test Results for Each Field's Feature Data:")
shapiro_results_df

Plate: 24278, Well: a01, ETC: 2m
Plate: 24278, Well: a02, ETC: 2m
Plate: 24278, Well: a03, ETC: 2m
Plate: 24278, Well: a04, ETC: 2m
Plate: 24278, Well: a05, ETC: 2m
Plate: 24278, Well: a06, ETC: 2m
Plate: 24278, Well: a07, ETC: 2m
Plate: 24278, Well: a08, ETC: 2m
Plate: 24278, Well: a09, ETC: 2m
Plate: 24278, Well: a10, ETC: 2m
Plate: 24278, Well: a11, ETC: 2m
Plate: 24278, Well: a12, ETC: 2m
Plate: 24278, Well: a13, ETC: 2m
Plate: 24278, Well: a14, ETC: 2m
Plate: 24278, Well: a15, ETC: 2m
Plate: 24278, Well: a16, ETC: 2m
Plate: 24278, Well: a17, ETC: 2m
Plate: 24278, Well: a18, ETC: 2m
Plate: 24278, Well: a19, ETC: 2m
Plate: 24278, Well: a20, ETC: 2m
Plate: 24278, Well: a21, ETC: 2m
Plate: 24278, Well: a22, ETC: 2m
Plate: 24278, Well: a23, ETC: 2m
Plate: 24278, Well: a24, ETC: 2m
Plate: 24278, Well: b01, ETC: 2m
Plate: 24278, Well: b02, ETC: 2m
Plate: 24278, Well: b03, ETC: 2m
Plate: 24278, Well: b04, ETC: 2m
Plate: 24278, Well: b05, ETC: 2m
Plate: 24278, Well: b06, ETC: 2m
Plate: 242

,PlateID,WellID,FieldID,Feature,Shapiro_Stat,Shapiro_p_value,Normality
0,24278,a01,s1,Cells_AreaShape_Area,0.524586,1.995330e-12,False
1,24278,a01,s1,Cells_AreaShape_Compactness,0.908840,3.564252e-04,False
2,24278,a01,s1,Nuclei_AreaShape_Area,0.462340,2.962369e-13,False
3,24278,a01,s1,Nuclei_AreaShape_Compactness,0.848451,3.766189e-06,False
4,24278,a01,s4,Cells_AreaShape_Area,0.501347,2.138397e-12,False
...,...,...,...,...,...,...,...
27287,24280,p24,s5,Nuclei_AreaShape_Compactness,0.814072,6.618734e-09,False
27288,24280,p24,s4,Cells_AreaShape_Area,0.932374,3.719401e-03,False
27289,24280,p24,s4,Cells_AreaShape_Compactness,0.796479,2.304788e-07,False
27290,24280,p24,s4,Nuclei_AreaShape_Area,0.819737,8.705807e-07,False


In [579]:
# subset the tests that result in not rejecting normality
true_normality = shapiro_results_df[shapiro_results_df['Normality']]
# group the normality counts by each plateID and feature
shapiro_summary1 = (true_normality.groupby(['PlateID', 'Feature'])
                    .size()
                    .unstack(fill_value=0))

# count the total number of fields for each plateID and add as a column
shapiro_summary1 = shapiro_summary1.merge(shapiro_results_df.groupby('PlateID')['FieldID'].count().reset_index(name='Field_Count'),
                                          on='PlateID',
                                          how='left')
shapiro_summary1 = move_column(shapiro_summary1, 'Field_Count', 1)

print("Shapiro-Wilk Test Results Summary:\nUnivariate Normality Counts per Feature")
shapiro_summary1

Shapiro-Wilk Test Results Summary:
Univariate Normality Counts per Feature


,PlateID,Field_Count,Cells_AreaShape_Area,Cells_AreaShape_Compactness,Nuclei_AreaShape_Area,Nuclei_AreaShape_Compactness
0,24278,9076,112,16,43,25
1,24279,9104,76,11,28,13
2,24280,9112,52,8,19,10


#### Multivariate Normality: Henze-Zirkler

The Henze-Zirkler test is more suited for testing multivariate normality across features simultaneously, making it ideal before applying MANOVA. It has a good overall power against alternatives to normality and works for any dimension and sample size.

In [551]:
# list to store MANOVA results
henze_results = []
lap_counter = 0

# parse through each well & plate combination
for plate_id, well_id in dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates().values:

    well_start_time = time.time()
    lap_counter += 1

    # subset the respective cell and nuclei data
    well_data_cells = dataframes['Cells'][(dataframes['Cells']['PlateID'] == plate_id) & 
                                          (dataframes['Cells']['WellID'] == well_id)]
    well_data_nuclei = dataframes['Nuclei'][(dataframes['Nuclei']['PlateID'] == plate_id) & 
                                            (dataframes['Nuclei']['WellID'] == well_id)]
    
    # merge cell and nuclei data with an outer join
    well_data = well_data_cells.merge(well_data_nuclei, on=['PlateID', 'WellID', 'FieldID', 'ImageNumber', 'ObjectNumber'], how='outer')
    
    # define the features to include in MANOVA and Mardia's test
    features = ['Cells_AreaShape_Area', 'Cells_AreaShape_Compactness', 'Nuclei_AreaShape_Area', 'Nuclei_AreaShape_Compactness']

    # ensure well_data has non-empty data for all required features before proceeding
    if well_data[features].dropna().shape[0] > 0:
        
        # iterate over each field within this well
        for field_id in well_data['FieldID'].unique():

            # subset the feature data for HZ's test
            field_data = well_data[well_data['FieldID'] == field_id][features].dropna()
            
            # only proceed with HZ's test if we have enough data
            if len(field_data) > 3:

                # execute HZ's multivariate normality test
                henze_test = pg.multivariate_normality(field_data, alpha=0.05)
                # extract normality result
                henze_stat = henze_test.hz
                henze_p_value = henze_test.pval
                henze_normal = henze_test.normal

                # append combined results to the list
                henze_results.append({
                    'PlateID': plate_id,
                    'WellID': well_id,
                    'FieldID': field_id,
                    'Henze_stat': henze_stat,
                    'Henze_p_value': henze_p_value,
                    'Normality': henze_normal,
                })
    else:
        print(f"Skipping henze-wirkler test for PlateID: {plate_id}, WellID: {well_id} due to insufficient data.")
    
    well_end_time = time.time() - well_start_time
    print(f"Plate: {plate_id}, Well: {well_id}, ETC: {(well_end_time * (len(dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates())
                                                                         - lap_counter)) / 60:.0f} minutes")

# convert results to dataframe
henze_results_df = pd.DataFrame(henze_results)
henze_results_df

Plate: 24278, Well: a01, ETC: 2 minutes
Plate: 24278, Well: a02, ETC: 2 minutes
Plate: 24278, Well: a03, ETC: 2 minutes
Plate: 24278, Well: a04, ETC: 2 minutes
Plate: 24278, Well: a05, ETC: 2 minutes
Plate: 24278, Well: a06, ETC: 2 minutes
Plate: 24278, Well: a07, ETC: 2 minutes
Plate: 24278, Well: a08, ETC: 2 minutes
Plate: 24278, Well: a09, ETC: 2 minutes
Plate: 24278, Well: a10, ETC: 2 minutes
Plate: 24278, Well: a11, ETC: 2 minutes
Plate: 24278, Well: a12, ETC: 2 minutes
Plate: 24278, Well: a13, ETC: 2 minutes
Plate: 24278, Well: a14, ETC: 2 minutes
Plate: 24278, Well: a15, ETC: 2 minutes
Plate: 24278, Well: a16, ETC: 2 minutes
Plate: 24278, Well: a17, ETC: 2 minutes
Plate: 24278, Well: a18, ETC: 2 minutes
Plate: 24278, Well: a19, ETC: 2 minutes
Plate: 24278, Well: a20, ETC: 2 minutes
Plate: 24278, Well: a21, ETC: 2 minutes
Plate: 24278, Well: a22, ETC: 2 minutes
Plate: 24278, Well: a23, ETC: 2 minutes
Plate: 24278, Well: a24, ETC: 2 minutes
Plate: 24278, Well: b01, ETC: 2 minutes


,PlateID,WellID,FieldID,Henze_stat,Henze_p_value,Normality
0,24278,a01,s1,2.968566,6.189672e-24,False
1,24278,a01,s2,1.410125,2.554677e-06,False
2,24278,a01,s3,1.258537,5.721446e-05,False
3,24278,a01,s4,3.555454,5.979282e-30,False
4,24278,a01,s5,0.883602,5.037093e-02,True
...,...,...,...,...,...,...
6818,24280,p24,s2,3.489236,3.846633e-32,False
6819,24280,p24,s3,3.902385,2.537495e-39,False
6820,24280,p24,s4,1.730779,8.750810e-10,False
6821,24280,p24,s5,5.576559,8.194577e-53,False


In [577]:
# create a short summary of henze-zirkler test results
henze_results_summary = henze_results_df.groupby('PlateID')['FieldID'].count().reset_index(name='Field_Count').merge(
    henze_results_df[henze_results_df['Normality']].groupby('PlateID').size().reset_index(name='Multivariate_Normality'),
    on='PlateID',
    how='left'
)

print(f"Henze-Zirkler Multivariate Test Results: Field Counts with MV Normality")
henze_results_summary

Henze-Zirkler Multivariate Test Results: Field Counts with MV Normality


,PlateID,Field_Count,Multivariate_Normality
0,24278,2269,19
1,24279,2276,10
2,24280,2278,11


### Non-parametric Testing

The results of univariate and multivariate normality tests show **very weak signs** of gaussian distribution. As a result it is better to continue with non-parametric tests that doesn't require normality pre-condition.

#### Kruskal-Wallis

A non-parametric test for differences across groups, suitable if you have multiple groups (fields).

In [631]:
# list to store kruskal-wallis results
kruskal_results = []
lap_counter = 0

# parse over each well & plate combination
for plate_id, well_id in dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates().values:

    well_start_time = time.time()
    lap_counter += 1
    
    # subset the respective cell and nuclei data
    well_data_cells = dataframes['Cells'][(dataframes['Cells']['PlateID'] == plate_id) & 
                                          (dataframes['Cells']['WellID'] == well_id)]
    well_data_nuclei = dataframes['Nuclei'][(dataframes['Nuclei']['PlateID'] == plate_id) & 
                                            (dataframes['Nuclei']['WellID'] == well_id)]
    
    # merge cell and nuclei data with outer join
    well_data = well_data_cells.merge(well_data_nuclei, on=['PlateID', 'WellID', 'FieldID', 'ImageNumber', 'ObjectNumber'], how='outer')
    
    features = ['Cells_AreaShape_Area', 'Cells_AreaShape_Compactness', 
                'Nuclei_AreaShape_Area', 'Nuclei_AreaShape_Compactness']
    
    # iterate over each feature
    for feature in features:

        # prepare data for the kruskal-wallis test across fields within the well
        groups = [well_data[well_data['FieldID'] == field][feature].dropna() for field in well_data['FieldID'].unique()]
        
        # proceed only if there’s enough data
        if all(len(group) > 3 for group in groups):
            # kurskal-wallis test
            stat, p_value = kruskal(*groups)
            
            kruskal_results.append({
                'PlateID': plate_id,
                'WellID': well_id,
                'Feature': feature,
                'Kruskal_Stat': stat,
                'Kruskal_p_value': p_value,
                'Heterogeneity': p_value < 0.05
            })
    
    well_end_time = time.time() - well_start_time
    print(f"Plate: {plate_id}, Well: {well_id}, ETC: {(well_end_time * (len(dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates())
                                                                         - lap_counter)) / 60:.0f}m")

kruskal_results_df = pd.DataFrame(kruskal_results)
kruskal_results_df

Plate: 24278, Well: a01, ETC: 2m
Plate: 24278, Well: a02, ETC: 3m
Plate: 24278, Well: a03, ETC: 2m
Plate: 24278, Well: a04, ETC: 2m
Plate: 24278, Well: a05, ETC: 2m
Plate: 24278, Well: a06, ETC: 2m
Plate: 24278, Well: a07, ETC: 2m
Plate: 24278, Well: a08, ETC: 2m
Plate: 24278, Well: a09, ETC: 2m
Plate: 24278, Well: a10, ETC: 2m
Plate: 24278, Well: a11, ETC: 2m
Plate: 24278, Well: a12, ETC: 2m
Plate: 24278, Well: a13, ETC: 2m
Plate: 24278, Well: a14, ETC: 2m
Plate: 24278, Well: a15, ETC: 2m
Plate: 24278, Well: a16, ETC: 2m
Plate: 24278, Well: a17, ETC: 2m
Plate: 24278, Well: a18, ETC: 2m
Plate: 24278, Well: a19, ETC: 2m
Plate: 24278, Well: a20, ETC: 2m
Plate: 24278, Well: a21, ETC: 2m
Plate: 24278, Well: a22, ETC: 2m
Plate: 24278, Well: a23, ETC: 2m
Plate: 24278, Well: a24, ETC: 2m
Plate: 24278, Well: b01, ETC: 2m
Plate: 24278, Well: b02, ETC: 2m
Plate: 24278, Well: b03, ETC: 3m
Plate: 24278, Well: b04, ETC: 2m
Plate: 24278, Well: b05, ETC: 2m
Plate: 24278, Well: b06, ETC: 2m
Plate: 242

,PlateID,WellID,Feature,Kruskal_Stat,Kruskal_p_value,Heterogeneity
0,24278,a01,Cells_AreaShape_Area,5.781568,3.280569e-01,False
1,24278,a01,Cells_AreaShape_Compactness,2.027772,8.452891e-01,False
2,24278,a01,Nuclei_AreaShape_Area,3.431292,6.338086e-01,False
3,24278,a01,Nuclei_AreaShape_Compactness,3.536971,6.178009e-01,False
4,24278,a02,Cells_AreaShape_Area,31.407216,7.783263e-06,True
...,...,...,...,...,...,...
4519,24280,p23,Nuclei_AreaShape_Compactness,16.727017,5.047698e-03,True
4520,24280,p24,Cells_AreaShape_Area,144.003630,2.520019e-29,True
4521,24280,p24,Cells_AreaShape_Compactness,7.529101,1.841687e-01,False
4522,24280,p24,Nuclei_AreaShape_Area,14.361427,1.346947e-02,True


In [667]:
def get_kruskal_summary(df):

    # total number of wells per plate
    wells_per_plate = df.groupby('PlateID')['WellID'].nunique()
    
    # table of significant tests
    significance_summary = df.pivot_table(
        index='PlateID',
        columns='Feature',
        values='Heterogeneity').round(2)
    
    # add total wells column
    significance_summary['Well_Count'] = wells_per_plate
    
    # put well count first
    significance_summary = move_column(significance_summary, 'Well_Count', 0)
    
    return significance_summary

plate_feature_summary = get_kruskal_summary(kruskal_results_df)
print(f"Percentage of Significant Well-Heterogeneity per Feature by Plate:")
plate_feature_summary

Percentage of Significant Well-Heterogeneity per Feature by Plate:


Feature,Well_Count,Cells_AreaShape_Area,Cells_AreaShape_Compactness,Nuclei_AreaShape_Area,Nuclei_AreaShape_Compactness
PlateID,,,,,
24278,375,0.94,0.38,0.53,0.29
24279,377,0.95,0.37,0.56,0.34
24280,379,0.95,0.33,0.54,0.28


#### Permutational Manova

Permutational multivariate analysis of variance (PERMANOVA), is a non-parametric multivariate statistical permutation test that is used to compare groups of objects and test the null hypothesis that the centroids and dispersion of the groups as defined by measure space are equivalent for all groups.

In [583]:
import numpy as np
import pandas as pd
import time
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
import itertools
from scipy.stats import f_oneway

def permanova_test(X, groups, n_perms=999):
    """
    Perform PERMANOVA test on the data
    
    Parameters:
    X: array-like, shape (n_samples, n_features)
        The multivariate data
    groups: array-like, shape (n_samples,)
        Group labels for each sample
    n_perms: int
        Number of permutations for the test
    
    Returns:
    F_stat: float
        The F statistic
    p_value: float
        The p-value from permutation test
    """
    # Calculate distance matrix
    dist_matrix = squareform(pdist(X, metric='euclidean'))
    
    # Calculate actual F statistic
    F_stat = _calculate_f_stat(dist_matrix, groups)
    
    # Permutation test
    perm_F_stats = []
    for _ in range(n_perms):
        perm_groups = np.random.permutation(groups)
        perm_F = _calculate_f_stat(dist_matrix, perm_groups)
        perm_F_stats.append(perm_F)
    
    # Calculate p-value
    p_value = sum(perm_F_stats >= F_stat) / (n_perms + 1)
    
    return F_stat, p_value

def _calculate_f_stat(dist_matrix, groups):
    """Helper function to calculate F statistic for PERMANOVA"""
    unique_groups = np.unique(groups)
    n = len(groups)
    
    # Calculate total sum of squares
    total_ss = np.sum(dist_matrix ** 2) / (2 * n)
    
    # Calculate within-group sum of squares
    within_ss = 0
    for group in unique_groups:
        mask = groups == group
        if sum(mask) > 1:  # Only calculate if group has more than one sample
            group_dist = dist_matrix[mask][:, mask]
            within_ss += np.sum(group_dist ** 2) / (2 * sum(mask))
    
    # Calculate F statistic
    between_ss = total_ss - within_ss
    df_between = len(unique_groups) - 1
    df_within = n - len(unique_groups)
    
    F_stat = (between_ss / df_between) / (within_ss / df_within)
    return F_stat

# list to store PERMANOVA results
permanova_results = []
lap_counter = 0

# parse over each well & plate combination
for plate_id, well_id in dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates().values:
    well_start_time = time.time()
    lap_counter += 1
    
    # subset the respective cell and nuclei data
    well_data_cells = dataframes['Cells'][
        (dataframes['Cells']['PlateID'] == plate_id) & 
        (dataframes['Cells']['WellID'] == well_id)
    ]
    well_data_nuclei = dataframes['Nuclei'][
        (dataframes['Nuclei']['PlateID'] == plate_id) & 
        (dataframes['Nuclei']['WellID'] == well_id)
    ]
    
    # merge cell and nuclei data with outer join
    well_data = well_data_cells.merge(
        well_data_nuclei,
        on=['PlateID', 'WellID', 'FieldID', 'ImageNumber', 'ObjectNumber'],
        how='outer'
    )
    
    features = [
        'Cells_AreaShape_Area',
        'Cells_AreaShape_Compactness',
        'Nuclei_AreaShape_Area',
        'Nuclei_AreaShape_Compactness'
    ]
    
    # Prepare data for PERMANOVA
    X = well_data[features].dropna()
    if len(X) > 3:  # Ensure enough data points
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Get group labels (FieldID)
        groups = well_data.loc[X.index, 'FieldID']
        
        if len(np.unique(groups)) > 1:  # Ensure more than one group
            # Perform PERMANOVA
            F_stat, p_value = permanova_test(X_scaled, groups, n_perms=999)
            
            permanova_results.append({
                'PlateID': plate_id,
                'WellID': well_id,
                'F_statistic': F_stat,
                'p_value': p_value,
                'Significance': p_value < 0.05
            })
    
    well_end_time = time.time() - well_start_time
    print(f"Plate: {plate_id}, Well: {well_id}, ETC: {(well_end_time * (len(dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates()) - lap_counter)) / 60:.0f}m")

permanova_results_df = pd.DataFrame(permanova_results)
permanova_results_df

Plate: 24278, Well: a01, ETC: 9m
Plate: 24278, Well: a02, ETC: 17m
Plate: 24278, Well: a03, ETC: 18m
Plate: 24278, Well: a04, ETC: 8m
Plate: 24278, Well: a05, ETC: 10m
Plate: 24278, Well: a06, ETC: 14m
Plate: 24278, Well: a07, ETC: 13m
Plate: 24278, Well: a08, ETC: 21m
Plate: 24278, Well: a09, ETC: 16m
Plate: 24278, Well: a10, ETC: 13m
Plate: 24278, Well: a11, ETC: 14m
Plate: 24278, Well: a12, ETC: 13m
Plate: 24278, Well: a13, ETC: 17m
Plate: 24278, Well: a14, ETC: 19m
Plate: 24278, Well: a15, ETC: 16m
Plate: 24278, Well: a16, ETC: 16m
Plate: 24278, Well: a17, ETC: 14m
Plate: 24278, Well: a18, ETC: 15m
Plate: 24278, Well: a19, ETC: 21m
Plate: 24278, Well: a20, ETC: 24m
Plate: 24278, Well: a21, ETC: 5m
Plate: 24278, Well: a22, ETC: 18m
Plate: 24278, Well: a23, ETC: 13m
Plate: 24278, Well: a24, ETC: 24m
Plate: 24278, Well: b01, ETC: 14m
Plate: 24278, Well: b02, ETC: 25m
Plate: 24278, Well: b03, ETC: 20m
Plate: 24278, Well: b04, ETC: 23m
Plate: 24278, Well: b05, ETC: 18m
Plate: 24278, Wel

KeyboardInterrupt: 

### Post-hoc Pairwise Testing

- When ANOVA shows significant differences among the sites of a well, it only tells: **there are significant differences between some of the sites.** It doesn't tell you which sites is behaving differently from the rest of the sites.
- When MANOVA shows significant differences among the sites of a well, it only tells you: **there are significant differences across the combination of dependent variables among some of the sites.** It doesn’t tell you which specific sites are behaving differently from the rest, nor does it indicate which particular dependent variables are driving those differences.
- Tukey’s HSD helps identify the specific groups (pairs) with statistically significant differences.

#### Dunn's Test

In [655]:
# list to store Kruskal-Wallis and Dunn's test results
kruskal_dunn_results = []
lap_counter = 0

# parse through each well & plate combination
for plate_id, well_id in dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates().values:

    well_start_time = time.time()
    lap_counter += 1

    # subset the respective cell and nuclei data
    well_data_cells = dataframes['Cells'][(dataframes['Cells']['PlateID'] == plate_id) & 
                                          (dataframes['Cells']['WellID'] == well_id)]
    well_data_nuclei = dataframes['Nuclei'][(dataframes['Nuclei']['PlateID'] == plate_id) & 
                                            (dataframes['Nuclei']['WellID'] == well_id)]
    
    # merge cell and nuclei data with outer join
    well_data = well_data_cells.merge(well_data_nuclei, on=['PlateID', 'WellID', 'FieldID', 'ImageNumber', 'ObjectNumber'], how='outer')
    
    # subset features to include in Kruskal-Wallis
    feature_columns = ['Cells_AreaShape_Area', 'Cells_AreaShape_Compactness', 'Nuclei_AreaShape_Area', 'Nuclei_AreaShape_Compactness']
    
    # ensure there are multiple fields to compare
    if len(well_data['FieldID'].unique()) > 1:
        
        # iterate over each feature and perform Kruskal-Wallis test
        for feature in feature_columns:
            
            # group data by FieldID for the current feature
            field_groups = [group[feature].dropna() for _, group in well_data.groupby('FieldID')]
            
            # perform Kruskal-Wallis test if each group has enough data points
            if all(len(group) > 1 for group in field_groups):
                kruskal_stat, kruskal_p_value = kruskal(*field_groups)
                
                # if Kruskal-Wallis is significant, proceed with Dunn's test
                if kruskal_p_value < 0.05:
                    # prepare data for Dunn's test by dropping any rows with missing values
                    dunn_data = well_data[['FieldID', feature]].dropna()
                    
                    # perform Dunn’s test for the current feature
                    dunn_results = posthocs.posthoc_dunn(dunn_data, val_col=feature, group_col='FieldID', p_adjust='bonferroni')
                    
                    # store results for each unique pairwise comparison (skip reverse duplicates)
                    for (group1, group2), p_adj in dunn_results.stack().items():
                        # Sort groups alphabetically and check if we're processing only unique pairs
                        if group1 < group2:  # ensures only unique pairs are processed
                            kruskal_dunn_results.append({
                                'PlateID': plate_id,
                                'WellID': well_id,
                                'Feature': feature,
                                'Group1': group1,
                                'Group2': group2,
                                'Kruskal_p_value': kruskal_p_value,
                                'Dunn_p_adj': p_adj,
                                'Reject': p_adj < 0.05
                            })

    well_end_time = time.time() - well_start_time
    print(f"Plate: {plate_id}, Well: {well_id}, ETC: {(well_end_time * (len(dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates()) - lap_counter)) / 60:.0f}m")

# convert results to dataframe
kruskal_dunn_results_df = pd.DataFrame(kruskal_dunn_results)
kruskal_dunn_results_df

Plate: 24278, Well: a01, ETC: 2m
Plate: 24278, Well: a02, ETC: 2m
Plate: 24278, Well: a03, ETC: 2m
Plate: 24278, Well: a04, ETC: 2m
Plate: 24278, Well: a05, ETC: 2m
Plate: 24278, Well: a06, ETC: 2m
Plate: 24278, Well: a07, ETC: 2m
Plate: 24278, Well: a08, ETC: 2m
Plate: 24278, Well: a09, ETC: 2m
Plate: 24278, Well: a10, ETC: 2m
Plate: 24278, Well: a11, ETC: 2m
Plate: 24278, Well: a12, ETC: 2m
Plate: 24278, Well: a13, ETC: 2m
Plate: 24278, Well: a14, ETC: 2m
Plate: 24278, Well: a15, ETC: 2m
Plate: 24278, Well: a16, ETC: 2m
Plate: 24278, Well: a17, ETC: 2m
Plate: 24278, Well: a18, ETC: 2m
Plate: 24278, Well: a19, ETC: 2m
Plate: 24278, Well: a20, ETC: 2m
Plate: 24278, Well: a21, ETC: 2m
Plate: 24278, Well: a22, ETC: 2m
Plate: 24278, Well: a23, ETC: 2m
Plate: 24278, Well: a24, ETC: 2m
Plate: 24278, Well: b01, ETC: 2m
Plate: 24278, Well: b02, ETC: 2m
Plate: 24278, Well: b03, ETC: 2m
Plate: 24278, Well: b04, ETC: 2m
Plate: 24278, Well: b05, ETC: 2m
Plate: 24278, Well: b06, ETC: 2m
Plate: 242

,PlateID,WellID,Feature,Group1,Group2,Kruskal_p_value,Dunn_p_adj,Reject
0,24278,a02,Cells_AreaShape_Area,s1,s2,0.000008,0.027333,True
1,24278,a02,Cells_AreaShape_Area,s1,s3,0.000008,0.000174,True
2,24278,a02,Cells_AreaShape_Area,s1,s4,0.000008,0.000216,True
3,24278,a02,Cells_AreaShape_Area,s1,s5,0.000008,0.009177,True
4,24278,a02,Cells_AreaShape_Area,s1,s6,0.000008,0.032776,True
...,...,...,...,...,...,...,...,...
36508,24280,p24,Nuclei_AreaShape_Area,s3,s5,0.013469,0.016862,True
36509,24280,p24,Nuclei_AreaShape_Area,s3,s6,0.013469,1.000000,False
36510,24280,p24,Nuclei_AreaShape_Area,s4,s5,0.013469,1.000000,False
36511,24280,p24,Nuclei_AreaShape_Area,s4,s6,0.013469,1.000000,False


Dunn's Test Summary:

In [708]:
kruskal_dunn_summary = kruskal_dunn_results_df.groupby('Feature')['Reject'].count().reset_index(name='Test_Count')
kruskal_dunn_summary['Pc_Rejection'] = kruskal_dunn_results_df.groupby('Feature')['Reject'].sum().reset_index(name='Pc_Rejection')['Pc_Rejection'] / kruskal_dunn_summary['Test_Count']

# rejection counts per feature and field
field_rejection_counts = kruskal_dunn_results_df[kruskal_dunn_results_df['Reject']].groupby(
    ['Feature', 'Group1']).size().reset_index(name='Rejection_Count')

# proportion of rejections per field within each feature
field_rejection_counts['Total_Rejections_Per_Feature'] = field_rejection_counts.groupby('Feature')['Rejection_Count'].transform('sum')
field_rejection_counts['Rejection_Share'] = field_rejection_counts['Rejection_Count'] / field_rejection_counts['Total_Rejections_Per_Feature']

# calculate HHI (sum of squared rejection shares) for each feature
hhi_per_feature = field_rejection_counts.groupby('Feature').apply(
    lambda x: (x['Rejection_Share'] ** 2).sum(), include_groups=False).reset_index(name='Rejection_HHI')

# merge HHI with the summary dataframe
kruskal_dunn_summary = kruskal_dunn_summary.merge(hhi_per_feature, on='Feature')

# Display the updated summary with HHI as the concentration metric
print("Kruskal Dunn Summary with Rejection Concentration:")
kruskal_dunn_summary

Kruskal Dunn Summary with Rejection Concentration:


,Feature,Test_Count,Pc_Rejection,Rejection_HHI
0,Cells_AreaShape_Area,16034,0.399214,0.245463
1,Cells_AreaShape_Compactness,6103,0.107488,0.247198
2,Nuclei_AreaShape_Area,9216,0.140951,0.242761
3,Nuclei_AreaShape_Compactness,5160,0.103101,0.243492


#### Tukey's HSD

In [726]:
# list to store MANOVA and Tukey's HSD results
manova_tukey_results = []
lap_counter = 0

# parse through each well & plate combination
for plate_id, well_id in dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates().values:

    well_start_time = time.time()
    lap_counter += 1

    # subset the respective cell and nuclei data
    well_data_cells = dataframes['Cells'][(dataframes['Cells']['PlateID'] == plate_id) & 
                                          (dataframes['Cells']['WellID'] == well_id)]
    well_data_nuclei = dataframes['Nuclei'][(dataframes['Nuclei']['PlateID'] == plate_id) & 
                                            (dataframes['Nuclei']['WellID'] == well_id)]
    
    # merge cell and nuclei data with outer join
    well_data = well_data_cells.merge(well_data_nuclei, on=['PlateID', 'WellID', 'FieldID', 'ImageNumber', 'ObjectNumber'], how='outer')
    
    # subset features to include in MANOVA
    feature_columns = ['Cells_AreaShape_Area', 'Cells_AreaShape_Compactness', 'Nuclei_AreaShape_Area', 'Nuclei_AreaShape_Compactness']
    
    # check if there is enough data to perform MANOVA
    if len(well_data['FieldID'].unique()) > 1:
        
        # execute Mardia's multivariate normality test
        field_data = well_data[feature_columns].dropna()
        mardia_test = pg.multivariate_normality(field_data, alpha=0.05)

        # proceed to MANOVA only if multivariate normality is met
        if mardia_test[0]:  # TRUE if normal

            # print(f"{plate_id} {well_id} Normality exist")
            
            # perform MANOVA across fields
            manova = MANOVA.from_formula(' + '.join(feature_columns) + ' ~ FieldID', data=well_data)
            manova_result = manova.mv_test()
            manova_p_value = manova_result.results['FieldID']['stat']['Pr > F'].iloc[0]
            
            # if MANOVA is significant, perform Tukey’s HSD for each feature
            if manova_p_value < 0.05:

                for feature in feature_columns:

                    # print(f"{plate_id} {well_id} {feature}")

                    # prepare data for Tukey's test by dropping any rows with missing values in either column
                    tukey_data = well_data[['FieldID', feature]].dropna()
                    
                    # perform Tukey's HSD for the current feature
                    tukey = pairwise_tukeyhsd(endog=tukey_data[feature],     # dependent variable
                                              groups=tukey_data['FieldID'],  # grouping by field
                                              alpha=0.05)
                    
                    # add results of each pairwise comparison test
                    for comparison in tukey.summary().data[1:]:  # skip header row
                        manova_tukey_results.append({
                            'PlateID': plate_id,
                            'WellID': well_id,
                            'Feature': feature,
                            'Group1': comparison[0],
                            'Group2': comparison[1],
                            'Mean Diff': comparison[2],
                            'p-adj': comparison[3],
                            'Reject': comparison[6],
                            'MANOVA_p_value': manova_p_value})
                        
    well_end_time = time.time() - well_start_time
    print(f"Plate: {plate_id}, Well: {well_id}, ETC: {(well_end_time * (len(dataframes['Cells'][['PlateID', 'WellID']].drop_duplicates())
                                                                         - lap_counter)) / 60:.0f}m")

# convert results to dataframe
manova_tukey_results_df = pd.DataFrame(manova_tukey_results)
manova_tukey_results_df

In [238]:
for plate_id in manova_tukey_results_df['PlateID'].drop_duplicates():
    print(f'PlateID: {plate_id}')
    print(f'Total well count: {len(dataframes['Image'][dataframes['Image']['PlateID'] == plate_id]['WellID'].drop_duplicates())}')
    print(f'Well count with significant heterogeneoity: {len(manova_tukey_results_df[manova_tukey_results_df['PlateID'] == plate_id]['WellID'].drop_duplicates())}')
    print(f'Total field count: {len(dataframes['Image'][dataframes['Image']['PlateID'] == plate_id][['PlateID', 'WellID', 'FieldID']].drop_duplicates())}')
    print(f'Field count with significant heterogeneoity: {len(manova_tukey_results_df[manova_tukey_results_df['PlateID'] == plate_id][['WellID', 'Group1']].drop_duplicates())}')

PlateID: 24278
Total well count: 384
Well count with significant heterogeneoity: 371
Total field count: 2282
Field count with significant heterogeneoity: 1847
PlateID: 24279
Total well count: 384
Well count with significant heterogeneoity: 366
Total field count: 2285
Field count with significant heterogeneoity: 1830


In [247]:
# Filter for rows where Reject is True
rejected_combinations = manova_tukey_results_df[manova_tukey_results_df['Reject']]

# Count occurrences of Reject == True for each Group1-Group2 combination within each PlateID and WellID
reject_summary = (
    rejected_combinations
    .groupby(['PlateID', 'WellID', 'Group1', 'Group2'])
    .size()
    .reset_index(name='Reject_Count'))

# calculate the frequency of reject == True for each group1-group2 combination within each plateID and wellID
reject_summary['Reject_Frequency'] = reject_summary['Reject_Count'] / manova_tukey_results_df['Feature'].nunique()
reject_summary.sort_values(by='Reject_Frequency', ascending=False)

,PlateID,WellID,Group1,Group2,Reject_Count,Reject_Frequency
3192,24279,g14,s3,s4,4,1.00
3883,24279,l20,s1,s6,4,1.00
1000,24278,h03,s3,s5,4,1.00
4295,24279,o16,s1,s6,4,1.00
885,24278,g05,s3,s4,4,1.00
...,...,...,...,...,...,...
2035,24278,o13,s3,s6,1,0.25
2036,24278,o14,s1,s2,1,0.25
2037,24278,o14,s2,s5,1,0.25
2038,24278,o14,s2,s6,1,0.25


In [254]:
# Step 1: Create bins for rejection frequency
# You can adjust these bin ranges as needed
bins = [0, 0.25, 0.5, 0.75, 1]  # e.g., 0-50% and 50-100%
labels = ['.25', '.5', '.75', '1.0']

# Assign each row in reject_summary to a frequency bin
reject_summary['Frequency_Bin'] = pd.cut(reject_summary['Reject_Frequency'], bins=bins, labels=labels, include_lowest=True)

# Step 2: Merge reject_summary with manova_tukey_results_df to include features
# Filter only rows where Reject is True, to include only rejected cases
rejected_features = manova_tukey_results_df[manova_tukey_results_df['Reject']].copy()

# Merge to add Feature column based on PlateID, WellID, Group1, and Group2
feature_reject_summary = pd.merge(
    reject_summary[['PlateID', 'WellID', 'Group1', 'Group2', 'Frequency_Bin']],
    rejected_features[['PlateID', 'WellID', 'Group1', 'Group2', 'Feature']],
    on=['PlateID', 'WellID', 'Group1', 'Group2'],
    how='left'
)

# Step 3: Count occurrences of each Feature within each Frequency_Bin
feature_distribution = feature_reject_summary.groupby(['Frequency_Bin', 'Feature']).size().unstack(fill_value=0)

# Display the final distribution table
print("Distribution of Rejected Features Across Frequency Bins:")
feature_distribution

Distribution of Rejected Features Across Frequency Bins:


/var/folders/vr/dsp7vw_n6rn1hrtlct_1zg9w0000gn/T/ipykernel_19515/2265388228.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  feature_distribution = feature_reject_summary.groupby(['Frequency_Bin', 'Feature']).size().unstack(fill_value=0)


Feature,Cells_AreaShape_Area,Cells_AreaShape_Compactness,Nuclei_AreaShape_Area,Nuclei_AreaShape_Compactness
Frequency_Bin,,,,
.25,3051,208,179,194
.5,745,405,286,146
.75,95,70,59,64
1.0,7,7,7,7


### Link Heterogeneity Results with Compound Data

In [724]:
# add compound information to heterogeneity results
heterogeneity_compound = kruskal_dunn_results_df.merge(
    biomorph[['PlateID', 'WellID', 'CPD_NAME']],
    on=['PlateID', 'WellID'],
    how='left')

# calculate the total number of comparisons for each compound
total_comparisons = (
    heterogeneity_compound
    .groupby('CPD_NAME')
    .size()
    .reset_index(name='Total_Comparisons'))

# calculate the number of significant site differences (Reject == True) for each compound
site_diff_counts = (
    heterogeneity_compound[heterogeneity_compound['Reject']]
    .groupby('CPD_NAME')
    .size()
    .reset_index(name='Site_Diff_Count'))

# merge total comparisons with site difference counts
compound_site_diff_summary = total_comparisons.merge(site_diff_counts, on='CPD_NAME', how='left')

# fill NaN values in Site_Diff_Count (for compounds with no rejections) with 0
compound_site_diff_summary['Site_Diff_Count'] = compound_site_diff_summary['Site_Diff_Count'].fillna(0)

# calculate the percentage of rejections
compound_site_diff_summary['Pc_Rejections'] = (
    compound_site_diff_summary['Site_Diff_Count'] / compound_site_diff_summary['Total_Comparisons'] * 100)

# sort by percentage descenting
print("Summary of Site Differences by Compound with Percentage of Rejections:")
compound_site_diff_summary.sort_values(by='Pc_Rejections', ascending=False, inplace=True)
compound_site_diff_summary


Summary of Site Differences by Compound with Percentage of Rejections:


,CPD_NAME,Total_Comparisons,Site_Diff_Count,Pc_Rejections
15,cabergoline,90,43,47.777778
56,mefenamic acid,105,47,44.761905
1,"2,4,5-trichlorophenoxyacetic acid",90,36,40.000000
40,fenretinide,90,36,40.000000
79,propranolol,75,29,38.666667
...,...,...,...,...
44,gw-9662,60,7,11.666667
89,tamoxifen,26,3,11.538462
60,mesulergine,75,6,8.000000
2,albendazole,75,6,8.000000


#### Analyze Consistency of Site Differences Across Wells

In [260]:
# Count significant site differences per compound and well
well_diff_consistency = (
    heterogeneity_compound[heterogeneity_compound['Reject']]
    .groupby(['CPD_NAME', 'WellID'])
    .size()
    .reset_index(name='Well_Site_Diff_Count')
)

# Calculate the percentage of wells with significant site differences for each compound
compound_well_diff_summary = (
    well_diff_consistency
    .groupby('CPD_NAME')
    .size()
    .reset_index(name='Wells_with_Site_Diff')
)

# Total well counts for each compound
total_wells_per_compound = biomorph.groupby('CPD_NAME')['WellID'].nunique().reset_index(name='Total_Well_Count')

# Merge to calculate the percentage of wells with site differences
compound_well_diff_summary = compound_well_diff_summary.merge(total_wells_per_compound, on='CPD_NAME')
compound_well_diff_summary['Site_Diff_Consistency'] = (compound_well_diff_summary['Wells_with_Site_Diff'] / compound_well_diff_summary['Total_Well_Count']) * 100

# Display the consistency summary
print("Consistency of Site Differences Across Wells by Compound:")
compound_well_diff_summary

Consistency of Site Differences Across Wells by Compound:


,CPD_NAME,Wells_with_Site_Diff,Total_Well_Count,Site_Diff_Consistency
0,"1,3,5-trimethoxybenzene",1,1,100.0
1,"2,4,5-trichlorophenoxyacetic acid",1,1,100.0
2,albendazole,1,1,100.0
3,ami-193,1,1,100.0
4,amiloride,1,1,100.0
...,...,...,...,...
93,tinidazole,1,1,100.0
94,tolbutamide,1,1,100.0
95,trapidil,1,1,100.0
96,triamterene,1,1,100.0


# 3. Classifier Modeling

- Since each endpoint represents a separate type of activity, building ***separate classifiers*** for each endpoint (like apoptosis up) is a reasonable approach. This way, each classifier can be optimized for the characteristics of its specific endpoint. However, if these endpoints are often activated together, a multi-label classification model could be beneficial, as it learns patterns across the endpoints simultaneously. **Next: The potential for multi-label will be investigated.**

- Including ***compound names*** could be useful since certain compounds are known to induce specific responses, but encoding must be done carefully. **Next: Categorical data encoding will be investigated.**
- Scaling using ***StandardScaler*** is done to mitigate potential sensitivity problems that might occur in feature-scale-sensitive model types. Random forest is not one of them, but still it is good practice.
- Since each WellID has multiple FieldIDs, treating them as groups using ***StratifiedGroupKFold*** ensures that fields from the same well don’t appear in both training and test sets, preventing data leakage and preserving the independence of samples.

In [55]:
# list of predictor columns
predictor_columns = [
    'Number_of_Cells', 'Cells_AreaShape_Area_Mean', 'Cells_AreaShape_Area_Std',
    'Cells_AreaShape_Compactness_Mean', 'Cells_AreaShape_Compactness_Std',
    'Nuclei_AreaShape_Area_Mean', 'Nuclei_AreaShape_Area_Std',
    'Nuclei_AreaShape_Compactness_Mean', 'Nuclei_AreaShape_Compactness_Std'
]

X = biomorph[predictor_columns]
y = biomorph['apoptosis up']
groups = biomorph['WellID']  # grouping by WellID to ensure fields stay together in splits

# initialize StratifiedGroupKFold
skf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# initialize random forest pipeline with scaling included
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('classifier', RandomForestClassifier(random_state=42))])

# list of scores for evaluation
accuracies = []
roc_aucs = []

# start the stratification of group cross-validation
for train_index, test_index in skf.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit the pipeline (that includes both scaler and classifier) on training data
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    accuracies.append(accuracy)
    roc_aucs.append(roc_auc)

print(f"Accuracy List: {accuracies}")
print(f"ROC AUC List: {roc_aucs}")

print(f"Mean Accuracy: {sum(accuracies) / len(accuracies):.2f}")
print(f"Mean ROC AUC: {sum(roc_aucs) / len(roc_aucs):.2f}")

Accuracy List: [0.6293103448275862, 0.6916666666666667, 0.7017543859649122, 0.7142857142857143, 0.8245614035087719]
ROC AUC List: [0.5326704545454546, 0.5659722222222221, 0.6282051282051282, 0.5429526748971193, 0.5367476851851851]
Mean Accuracy: 0.71
Mean ROC AUC: 0.56
